<a href="https://colab.research.google.com/github/carlos-dani-dev/carlos-dani-dev/blob/main/notebooks/Operacao_8_Vectorizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Monte o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Download do dataset
!gdown --id 1_Elg5O_H0fJ1mIWEKkb6q4ul9x7SIhnk -O /content/drive/MyDrive/pln_data/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1_Elg5O_H0fJ1mIWEKkb6q4ul9x7SIhnk
To: /content/drive/MyDrive/pln_data/b2w_reviews_samsung_cleaned.csv
100% 801k/801k [00:00<00:00, 39.4MB/s]


In [2]:
!pip install gensim
# Executar no início dos notebooks
!pip install gensim sentence-transformers
!pip install wordcloud plotly

# Para downloads do dataset
!pip install gdown

In [5]:
# Verificar uso de RAM
!cat /proc/meminfo | head -n 3

# Verificar GPU disponível
!nvidia-smi

MemTotal:       13289424 kB
MemFree:         9364584 kB
MemAvailable:   12228300 kB
/bin/bash: line 1: nvidia-smi: command not found


In [8]:
# Bibliotecas principais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Pré-processamento
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [1]:
# TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Word Embeddings
import gensim
from gensim.models import Word2Vec, Doc2Vec
from sentence_transformers import SentenceTransformer

# NLTK
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
reviews = pd.read_csv('/content/drive/MyDrive/PLN_B2W_Reviews/data/processed/b2w_reviews_samsung_modified.csv')
reviews

,review_text,overall_rating,recommend_to_a_friend
0,Uma tela impecável. Se sua prioridade é tela e...,4,1
1,Melhor custo benefício. Exatamente como anunc...,5,1
2,"Muito útil, para pesquisas e baixar publicaçõe...",5,1
3,A entrega sempre no prazo e muitas vezes até a...,4,1
4,"Recomendo tanto o produto quanto a loja, produ...",4,1
...,...,...,...
5748,O produto é muito bom. O designer é perfeito. ...,5,1
5749,"Excelente produto. Acima do esperado, Recomend...",5,1
5750,"Amei o produto muito bom ,chegou bem antes do ...",5,1
5751,"Qualidade de imagem e som são excelentes, além...",5,1


In [13]:
all_reviews = reviews['review_text']
all_reviews
#

,review_text
0,Uma tela impecável. Se sua prioridade é tela e...
1,Melhor custo benefício. Exatamente como anunc...
2,"Muito útil, para pesquisas e baixar publicaçõe..."
3,A entrega sempre no prazo e muitas vezes até a...
4,"Recomendo tanto o produto quanto a loja, produ..."
...,...
5748,O produto é muito bom. O designer é perfeito. ...
5749,"Excelente produto. Acima do esperado, Recomend..."
5750,"Amei o produto muito bom ,chegou bem antes do ..."
5751,"Qualidade de imagem e som são excelentes, além..."


In [26]:
count_vect = CountVectorizer()
'''
count_vect = CountVectorizer(
    stop_words="portuguese",
    min_df=2,
    max_df=0.8
)
'''
X_count = count_vect.fit_transform(all_reviews)
X_count = X_count.toarray()
'''
print("Matriz CountVectorizer (bag of words):")
print(X_count)  # matriz densa com cada review vetorizada
print("Vocabulário (features):")
print(count_vect.get_feature_names_out())
'''
df = pd.DataFrame(X_count, columns=count_vect.get_feature_names_out())
print(df)

      00  000  01  015  02  03  04  05  06  07  ...  ùnico  última  último  \
0      0    0   0    0   0   0   0   0   0   0  ...      0       0       0   
1      0    0   0    0   0   0   0   0   0   0  ...      0       0       0   
2      0    0   0    0   0   0   0   0   0   0  ...      0       0       0   
3      0    0   0    0   0   0   0   0   0   0  ...      0       0       0   
4      0    0   0    0   0   0   0   0   0   0  ...      0       0       0   
...   ..  ...  ..  ...  ..  ..  ..  ..  ..  ..  ...    ...     ...     ...   
5748   0    0   0    0   0   0   0   0   0   0  ...      0       0       0   
5749   0    0   0    0   0   0   0   0   0   0  ...      0       0       0   
5750   0    0   0    0   0   0   0   0   0   0  ...      0       0       0   
5751   0    0   0    0   0   0   0   0   0   0  ...      0       0       0   
5752   0    0   0    0   0   0   0   0   0   0  ...      0       0       0   

      últimos  úmida  única  únicas  único  úteis  útil  
0    

In [27]:
tfidf_vect = TfidfVectorizer()
'''
tfidf_vect = TfidfVectorizer(
    stop_words="portuguese",
    min_df=2,
    max_df=0.8
)
'''
X_tfidf = tfidf_vect.fit_transform(all_reviews)
X_tfidf = X_tfidf.toarray()

'''
print("\nMatriz TF-IDF:")
print(X_tfidf.toarray())
print("Vocabulário (features):")
print(tfidf_vect.get_feature_names_out())
'''
df = pd.DataFrame(X_tfidf, columns=tfidf_vect.get_feature_names_out())
print(df)

       00  000   01  015   02   03   04   05   06   07  ...  ùnico  última  \
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0     0.0   
1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0     0.0   
2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0     0.0   
3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0     0.0   
4     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0     0.0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...    ...     ...   
5748  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0     0.0   
5749  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0     0.0   
5750  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0     0.0   
5751  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0     0.0   
5752  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0     0.0   

      último  últimos  úmida  única  únicas  único  úteis      